In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import random

np.random.seed(42)
random.seed(42)

def regularization(G, dim, embedding_feature: str = 'embedding', iterations=3):
    G = G.copy()

    nodes = []

    # inicializando vetor f para todos os nodes
    for node in G.nodes():
        G.nodes[node]['f'] = np.array([0.0]*dim)
        if embedding_feature in G.nodes[node]:
          G.nodes[node]['f'] = np.array(G.nodes[node][embedding_feature])*1.0 
        nodes.append(node)

    pbar = tqdm(range(0, iterations))

    for iteration in pbar:
        random.shuffle(nodes)
        energy = 0.0

        # percorrendo cada node
        for node in nodes:
            f_new = np.array([0.0]*dim)
            f_old = np.array(G.nodes[node]['f'])*1.0
            sum_w = 0.0

            # percorrendo vizinhos do onde
            w = 1
            for neighbor in G.neighbors(node):

                f_new += G.nodes[neighbor]['f']

                sum_w += G.degree[neighbor]

            f_new /= sum_w

            G.nodes[node]['f'] = f_new*1.0

            if embedding_feature in G.nodes[node]:
                G.nodes[node]['f'] = np.array(G.nodes[node][embedding_feature])

            G.nodes[node]['f_'+embedding_feature] = G.nodes[node]['f']
            energy += np.linalg.norm(f_new-f_old)

        iteration += 1
        message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
        pbar.set_description(message)

    return G

In [5]:
from google.colab import drive

drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [6]:
PATH_TO_PICKLE = "/content/drive/MyDrive/USP/Doctorate/Research/Articles/Early Fusion For One-class Learning On Heterogeneous Graphs/grafos"
PATH_TO_SAVE = "/content/drive/MyDrive/Multimodal_Fusion/grafos_reg"

In [7]:
import networkx as nx
import pickle as pkl
import os

pickles = os.listdir(PATH_TO_PICKLE)
pickles

['rec_sys_1.pkl',
 'rec_sys_2.pkl',
 'rec_sys_3.pkl',
 'rec_sys_4.pkl',
 'rec_sys_5.pkl',
 'event.pkl',
 'music.pkl',
 'fakenews.pkl']

In [ ]:
import networkx as nx
import pickle as pkl
import os

pickles = os.listdir(PATH_TO_PICKLE)

for pickle in pickles:
    with open(f"{PATH_TO_PICKLE}/{pickle}", "rb") as file:
        G = pkl.load(file)

    for node in G.nodes:
        if pickle == 'music.pkl' and G.nodes[node]['type'] == 'artist':
            del G.nodes[node]['features']

        if 'features' in G.nodes[node]:
            dim = G.nodes[node]['features'].shape[0]
    
    G = regularization(G, dim = dim, embedding_feature = 'features', iterations=1)
    
    with open(f"{PATH_TO_SAVE}/{pickle}", "wb") as file:
        pkl.dump(G, file)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]